In [5]:
import numpy as np
import json
from munch import Munch
import itertools
from collections import defaultdict
import random
import copy
import pickle
import importlib

import apt_helper as ahlp
from mv_Viterbi import mv_Viterbi

In [9]:
cst_names = ['know_sally_exists','have_sally_credential', 'learn_where_data_stored', 'have_data_on_ds', 'have_data_on_hi', 'have_data_on_he']

In [10]:

cst_list=  []
for name in cst_names:
    module = importlib.import_module(name)
    curr_cst =  Munch(name = module.name, \
                      aux_size = module.aug_size, \
                      update_fun = module.update_fun, \
                      init_fun = module.init_fun, \
                      cst_fun = module.cst_fun)
    if hasattr(module, 'dependency'):
        curr_cst.dependency = module.dependency
    cst_list.append(curr_cst)
    

    


# Know Sally Exists

In [30]:
def update_fun(k,r ,k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = tau^t. the hitting time of (DI, (HI, usr/query))
    m2^t = [1- (1 - tau^t_a) AND c)] AND m2^{t-1} = [tau^t_a or (1 - c)] AND m2^{t-1} #tracks if the arrival time of a is before c
    '''
    m1 = (k == ('DI',('HI','usr/query'))) or r_past[0] #tracks if knowledge state has occured yet
    forbidden_transitions = (k_past[0] == 'EX' and k[0] == 'CA') or (k_past[0] == 'DI' and k[0] == 'CA')
    forbidden_emissions = (k == ('CA',('HI','usr/query')))
    m2 = (m1 or (not (forbidden_transitions and forbidden_emissions))) and r_past[1] 
          
    #at the time of first hit, all forbidden transitions/emissions are impossible. current logical formulation ok.

    return int(r == (m1,m2))

def init_fun(k, r):
    '''
    initial "prob" of r = (m1,m2) from k. is just indicator
    '''
    m1 = k == ('DI',('HI','usr/query'))
    m2 = not (k == ('CA',('HI','usr/query'))) #at first time, can only violate the emission constraint.


    return int(r == (m1,m2))
    
def cst_fun(k,r, sat):
    '''
    Constraint is a boolean emissions of the final auxillary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat) 

In [31]:
know_sally_exists = Munch(name = 'know_sally_exists',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun)
apt_cst_list.append(know_sally_exists)

In [32]:
with open('know_sally_exists.pkl', 'wb') as f:
    pickle.dump(know_sally_exists, f)


# Have sally credential

In [33]:
def update_fun(k, r, k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = (k == ('CA', ('HI', 'usr/query'))) or r_past[0]  # tracks if the credential state has occurred yet
    forbidden_emissions = (k == ('EX', ('V', 'access/sally'))) or (k == ('EX', ('DS', 'syslog/nano'))) or (k == ('DI', ('DS', 'syslog/ls')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]
    '''
    m1 = (k == ('CA', ('HI', 'usr/query'))) or r_past[0]  # tracks if the credential state has occurred yet
    forbidden_emissions = (k == ('EX', ('V', 'access/sally'))) or (k == ('EX', ('DS', 'syslog/nano'))) or (k == ('DI', ('DS', 'syslog/ls')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]

    return int(r == (m1, m2))

def init_fun(k, r):
    '''
    Initial "prob" of r = (m1, m2) from k. Is just indicator
    '''
    m1 = k == ('CA', ('HI', 'usr/query'))
    m2 = not ((k == ('EX', ('V', 'access/sally'))) or (k == ('EX', ('DS', 'syslog/nano'))) or (k == ('DI', ('DS', 'syslog/ls'))))  # at first time, can only violate the emission constraint.

    return int(r == (m1, m2))

def cst_fun(k, r, sat):
    '''
    Constraint is a boolean emissions of the final auxiliary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat)


In [34]:
have_sally_credential = Munch(name = 'have_sally_credential',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun, dependency = 'know_sally_exists')
apt_cst_list.append(have_sally_credential)

In [35]:
with open('have_sally_credential.pkl', 'wb') as f:
    pickle.dump(have_sally_credential, f)


# Learn where data is stored

In [36]:
def update_fun(k, r, k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = (k == ('DI', ('DS', 'syslog/ls'))) or r_past[0]  # tracks if the data storage state has occurred yet
    forbidden_transitions = (k_past[0] == 'DI' and k[0] == 'COL')
    forbidden_emissions = (k == ('COL', ('DS', 'syslog/nano')))
    m2 = (m1 or (not (forbidden_transitions and forbidden_emissions))) and r_past[1]
    '''
    m1 = (k == ('DI', ('DS', 'syslog/ls'))) or r_past[0]  # tracks if the data storage state has occurred yet
    forbidden_transitions = (k_past[0] == 'DI' and k[0] == 'COL')
    forbidden_emissions = (k == ('COL', ('DS', 'syslog/nano')))
    m2 = (m1 or (not (forbidden_transitions and forbidden_emissions))) and r_past[1]

    return int(r == (m1, m2))

def init_fun(k, r):
    '''
    Initial "prob" of r = (m1, m2) from k. Is just indicator
    '''
    m1 = k == ('DI', ('DS', 'syslog/ls'))
    m2 = not (k == ('COL', ('DS', 'syslog/nano')))  # at first time, can only violate the emission constraint.

    return int(r == (m1, m2))

def cst_fun(k, r, sat):
    '''
    Constraint is a boolean emissions of the final auxiliary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat)


In [37]:
learn_where_data_stored = Munch(name = 'learn_where_data_stored',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun, dependency = 'have_sally_credential')
apt_cst_list.append(learn_where_data_stored)

In [38]:
with open('learn_where_data_stored.pkl', 'wb') as f:
    pickle.dump(learn_where_data_stored, f)


# Have data on ds

In [39]:
def update_fun(k, r, k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = (k == ('COL', ('DS', 'syslog/nano'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_emissions = (k == ('COL', ('HI', 'img/post')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]
    '''
    m1 = (k == ('COL', ('DS', 'syslog/nano'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_emissions = (k == ('COL', ('HI', 'img/post')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]

    return int(r == (m1, m2))

def init_fun(k, r):
    '''
    Initial "prob" of r = (m1, m2) from k. Is just indicator
    '''
    m1 = k == ('COL', ('DS', 'syslog/nano'))
    m2 = not (k == ('COL', ('HI', 'img/post')))  # at first time, can only violate the emission constraint.

    return int(r == (m1, m2))

def cst_fun(k, r, sat):
    '''
    Constraint is a boolean emissions of the final auxiliary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat)


In [40]:
have_data_on_ds = Munch(name = 'have_data_on_ds',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun, dependency = 'learn_where_data_stored')
apt_cst_list.append(have_data_on_ds)

In [41]:
with open('have_data_on_ds.pkl', 'wb') as f:
    pickle.dump(have_data_on_ds, f)


# Have data on hi

In [42]:
def update_fun(k, r, k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = (k == ('COL', ('HI', 'img/post'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_emissions = (k == ('COL', ('HE', 'img/post')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]
    '''
    m1 = (k == ('COL', ('HI', 'img/post'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_emissions = (k == ('COL', ('HE', 'img/post')))
    m2 = (m1 or not forbidden_emissions) and r_past[1]

    return int(r == (m1, m2))

def init_fun(k, r):
    '''
    Initial "prob" of r = (m1, m2) from k. Is just indicator
    '''
    m1 = k == ('COL', ('HI', 'img/post'))
    m2 = not (k == ('COL', ('HE', 'img/post')))  # at first time, can only violate the emission constraint.

    return int(r == (m1, m2))

def cst_fun(k, r, sat):
    '''
    Constraint is a boolean emissions of the final auxiliary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat)


In [43]:
have_data_on_hi = Munch(name = 'have_data_on_hi',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun, dependency = 'have_data_on_ds')
apt_cst_list.append(have_data_on_hi)

In [44]:
with open('have_data_on_hi.pkl', 'wb') as f:
    pickle.dump(have_data_on_hi, f)


# Have data on he

In [45]:
def update_fun(k, r, k_past, r_past):
    '''
    r^t = (m_1^t, m_2^t)
    m1^t = (k == ('COL', ('HE', 'img/post'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_transitions = (k_past[0] == 'COL' and k[0] == 'EXF')
    forbidden_emissions = (k == ('EXF', ('HE', 'img/query')))
    m2 = (m1 or (not (forbidden_transitions and forbidden_emissions))) and r_past[1]
    '''
    m1 = (k == ('COL', ('HE', 'img/post'))) or r_past[0]  # tracks if the data state has occurred yet
    forbidden_transitions = (k_past[0] == 'COL' and k[0] == 'EXF')
    forbidden_emissions = (k == ('EXF', ('HE', 'img/query')))
    m2 = (m1 or (not (forbidden_transitions and forbidden_emissions))) and r_past[1]

    return int(r == (m1, m2))

def init_fun(k, r):
    '''
    Initial "prob" of r = (m1, m2) from k. Is just indicator
    '''
    m1 = k == ('COL', ('HE', 'img/post'))
    m2 = not (k == ('EXF', ('HE', 'img/query')))  # at first time, can only violate the emission constraint.

    return int(r == (m1, m2))

def cst_fun(k, r, sat):
    '''
    Constraint is a boolean emissions of the final auxiliary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat)


In [46]:
have_data_on_he = Munch(name = 'have_data_on_he',aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun, dependency = 'have_data_on_hi')
apt_cst_list.append(have_data_on_he)

In [47]:
with open('have_data_on_he.pkl', 'wb') as f:
    pickle.dump(have_data_on_he, f)


In [ ]:
from munch import Munch

def create_updatefun(zip_list):
    def update_fun_agg(r,k,r_past):
        val = 1
        for cst, ix in zip_list:
            val *= cst.update_fun(tuple(r[ix[0]:ix[1]]),k,tuple(r_past[ix[0]:ix[1]]))
        return val
    return update_fun_agg

def create_initfun(zip_list):
    def init_fun_agg(k,r):
        val = 1
        for cst,ix in zip_list:
            val *= cst.init_fun(k,tuple(r[ix[0]:ix[1]]))
        return val
    return init_fun_agg

def create_cstfun(zip_list):
    def cst_fun_agg(r,sat):
        val = 1
        it = 0
        for cst,ix in zip_list:
            val*= cst.cst_fun(tuple(r[ix[0]:ix[1]]),sat[it])
            it += 1
        return val
    return cst_fun_agg

def apt_cst_aggregate(cst_list):
    names_list = [cst.names for cst in cst_list]
    ix_list = []
    name_list = []
    for cst in cst_list:
        r_ix = l_ix + cst.aux_size
        ix_list.append((l_ix,r_ix)) #tuple of indices of the aux stats that correspond to each state
        l_ix = r_ix
        name_list.append(cst.name)
    zip_list = list(zip(cst_list,ix_list))

    cst_combined = Munch(name = name_list, aux_size = r_ix, update_fun = create_updatefun(zip_list), \
                         init_fun = create_initfun(zip_list), cst_fun = create_cstfun(zip_list))
    
    return cst_combined

In [ ]:

def create_updatefun(zip_list, cst_ix):
    def update_fun_agg(k,r,k_past,r_past):
        val = 1
        for cst, ix in zip_list:
            val *= cst.update_fun(tuple(k,r[ix[0]:ix[1]]),k_past,tuple(r_past[ix[0]:ix[1]]))
            if cst.dependency:
                val *= int(r[cst.dependency] or (not r[cst.name])) #either the dependcy is satisifed or the current knowledge hasn't been attained.
        return val
    return update_fun_agg

def create_initfun(zip_list, cst_ix):
    def init_fun_agg(k,r):
        val = 1
        for cst,ix in zip_list:
            val *= cst.init_fun(k,tuple(r[ix[0]:ix[1]]))
        return val
    return init_fun_agg

def create_cstfun(zip_list, cst_ix):
    def cst_fun_agg(k,r,sat):
        val = 1
        it = 0
        for cst,ix in zip_list:
            val*= cst.cst_fun(k,tuple(r[ix[0]:ix[1]]),sat[it])
            it += 1
        return val
    return cst_fun_agg

def apt_cst_aggregate(cst_list):
    '''
    Assumes that the first Boolean of a constraint tracks whether the knowledge state has been hit.
    '''
    l_ix = 0
    r_ix = 0
    ix_list = []
    name_list = []
    cst_ix = {}
    for cst in cst_list:
        cst_ix[cst.name] = l_ix
        r_ix = l_ix + cst.aux_size
        ix_list.append((l_ix,r_ix)) #tuple of indices of the aux stats that correspond to each state
        l_ix = r_ix
    zip_list = list(zip(cst_list,ix_list))

    cst_combined = Munch(name = name_list, aux_size = r_ix, update_fun = create_updatefun(zip_list, cst_list), \
                         init_fun = create_initfun(zip_list, cst_list), cst_fun = create_cstfun(zip_list, cst_list))
    
    return cst_combined